In [ ]:
!pip install google-cloud-core
!pip install google-cloud-bigtable
!pip install python-time
!pip install datetime

In [1]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters

/usr/lib/python3.10/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 2.0.5-build-libtorrent-rasterbar-src-libtorrent-rasterbar-2.0.5-bindings-python is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'orders_{}'.format(time.time())

In [3]:
TABLE_ID

'orders_1649466418.1208806'

### Utils functions

In [4]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")

### Connecting to BigTable

In [5]:
client = bigtable.Client.from_service_account_json('../unbosque-service-account.json', admin = True)

In [6]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


### Creating a table

In [7]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the orders_1649466418.1208806 table


In [8]:
print('Creating all family columns')

customer_cfId = 'customer'
customer_cf = table.column_family(customer_cfId)
customer_cf.create()

items_cfId = 'items'
items_cf = table.column_family(items_cfId)
items_cf.create()

delivery_cfId = 'delivery'
delivery_cf = table.column_family(delivery_cfId)
delivery_cf.create()

Creating all family columns


### Inserting data

In [11]:
orders = [
    {
        'orderId': '2497',
        'customer': {
            'firstName': 'Fabian',
            'lastName': 'Penia',
            'customerId': '101'
        },
        'items': [
            {
                'productId': '3301',
                'quantity': '5',
                'unitPrice': '12000'
            },
            {
                'productId': '3302',
                'quantity': '1',
                'unitPrice': '18000'
            },
            {
                'productId': '3303',
                'quantity': '2',
                'unitPrice': '25000'
            }
        ],
        'delivery': {
            'addressLine1': 'Fake Street 123',
            'addressLine2': 'House 10',
            'city': 'Springfield',
            'country': 'US'
        }
    },
    {
        'orderId': '2498',
        'customer': {
            'firstName': 'Pepito',
            'lastName': 'Perez',
            'customerId': '102'
        },
        'items': [
            {
                'productId': '3301',
                'quantity': '1',
                'unitPrice': '12000'
            },
            {
                'productId': '3306',
                'quantity': '3',
                'unitPrice': '20000'
            }
        ],
        'delivery': {
            'addressLine1': '742 Evergreen Terrace',
            'city': 'Springfield',
            'country': 'US'
        }
    }
]

In [12]:
orders

[{'orderId': '2497',
  'customer': {'firstName': 'Fabian',
   'lastName': 'Penia',
   'customerId': '101'},
  'items': [{'productId': '3301', 'quantity': '5', 'unitPrice': '12000'},
   {'productId': '3302', 'quantity': '1', 'unitPrice': '18000'},
   {'productId': '3303', 'quantity': '2', 'unitPrice': '25000'}],
  'delivery': {'addressLine1': 'Fake Street 123',
   'addressLine2': 'House 10',
   'city': 'Springfield',
   'country': 'US'}},
 {'orderId': '2498',
  'customer': {'firstName': 'Pepito',
   'lastName': 'Perez',
   'customerId': '102'},
  'items': [{'productId': '3301', 'quantity': '1', 'unitPrice': '12000'},
   {'productId': '3306', 'quantity': '3', 'unitPrice': '20000'}],
  'delivery': {'addressLine1': '742 Evergreen Terrace',
   'city': 'Springfield',
   'country': 'US'}}]

In [ ]:
dt = dt.utcnow()
rows = []

print('Writing orders to the table')
for order in orders:
    
    row_key = 'order#{}'.format(order['orderId']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(customer_cfId, 'firstName'.encode(), order['customer']['firstName'], timestamp = dt)
    row.set_cell(customer_cfId, 'lastName'.encode(), order['customer']['lastName'], timestamp = dt)
    row.set_cell(customer_cfId, 'customerId'.encode(), order['customer']['customerId'], timestamp = dt)
    
    for item in order['items']:
        row.set_cell(items_cfId, 'product#{}'.format(item['productId']).encode(), (item['quantity'] + ':' + item['unitPrice']), timestamp = dt)
    
    row.set_cell(delivery_cfId, 'addressLine1'.encode(), order['delivery']['addressLine1'], timestamp = dt)
    try:
        row.set_cell(delivery_cfId, 'addressLine2'.encode(), order['delivery']['addressLine2'], timestamp = dt)
    except KeyError:
        pass
    row.set_cell(delivery_cfId, 'city'.encode(), order['delivery']['city'], timestamp = dt)
    row.set_cell(delivery_cfId, 'country'.encode(), order['delivery']['country'], timestamp = dt)
    
    rows.append(row)

table.mutate_rows(rows)

Writing orders to the table


[, ]

### Reading data

In [ ]:
print('Getting a single order by row key')
key = 'order#2497'.encode()

row = table.read_row(key)
print_row(row)

Getting a single order by row key
Reading data for order#2497:
Column Family customer
	customerId: 101 @2022-04-08 00:53:00.990000+00:00
	customerId: 101 @2022-04-08 00:52:45.459000+00:00
	firstName: Fabian @2022-04-08 00:53:00.990000+00:00
	firstName: Fabian @2022-04-08 00:52:45.459000+00:00
	lastName: Penia @2022-04-08 00:53:00.990000+00:00
	lastName: Penia @2022-04-08 00:52:45.459000+00:00
Column Family delivery
	addressLine1: Fake Street 123 @2022-04-08 00:53:00.990000+00:00
	addressLine1: Fake Street 123 @2022-04-08 00:52:45.459000+00:00
	addressLine2: House 10 @2022-04-08 00:53:00.990000+00:00
	addressLine2: House 10 @2022-04-08 00:52:45.459000+00:00
	city: Springfield @2022-04-08 00:53:00.990000+00:00
	city: Springfield @2022-04-08 00:52:45.459000+00:00
	country: US @2022-04-08 00:53:00.990000+00:00
	country: US @2022-04-08 00:52:45.459000+00:00
Column Family items
	product#3301: 5:12000 @2022-04-08 00:53:00.990000+00:00
	product#3301: 5:12000 @2022-04-08 00:52:45.459000+00:00
	

In [ ]:
print('Getting a specific product item by row key')
key = 'order#2497'.encode()

col_filter = row_filters.ColumnQualifierRegexFilter('product#3302'.encode())

row = table.read_row(key, filter_ = col_filter)
print_row(row)

Getting a specific product item by row key
Reading data for order#2497:
Column Family items
	product#3302: 1:18000 @2022-04-08 00:53:00.990000+00:00
	product#3302: 1:18000 @2022-04-08 00:52:45.459000+00:00



In [ ]:
print('Getting all orders')

rows = table.read_rows()
for row in rows:
    print_row(row)

Getting all orders
Reading data for order#2497:
Column Family customer
	customerId: 101 @2022-04-08 00:53:00.990000+00:00
	customerId: 101 @2022-04-08 00:52:45.459000+00:00
	firstName: Fabian @2022-04-08 00:53:00.990000+00:00
	firstName: Fabian @2022-04-08 00:52:45.459000+00:00
	lastName: Penia @2022-04-08 00:53:00.990000+00:00
	lastName: Penia @2022-04-08 00:52:45.459000+00:00
Column Family delivery
	addressLine1: Fake Street 123 @2022-04-08 00:53:00.990000+00:00
	addressLine1: Fake Street 123 @2022-04-08 00:52:45.459000+00:00
	addressLine2: House 10 @2022-04-08 00:53:00.990000+00:00
	addressLine2: House 10 @2022-04-08 00:52:45.459000+00:00
	city: Springfield @2022-04-08 00:53:00.990000+00:00
	city: Springfield @2022-04-08 00:52:45.459000+00:00
	country: US @2022-04-08 00:53:00.990000+00:00
	country: US @2022-04-08 00:52:45.459000+00:00
Column Family items
	product#3301: 5:12000 @2022-04-08 00:53:00.990000+00:00
	product#3301: 5:12000 @2022-04-08 00:52:45.459000+00:00
	product#3302: 1

In [ ]:
print('Getting a specific column for all orders')

col_filter = row_filters.ColumnQualifierRegexFilter('product#3302'.encode())
        
rows = table.read_rows(filter_ = col_filter)
for row in rows:
    print_row(row)

Getting a specific column for all orders
Reading data for order#2497:
Column Family items
	product#3302: 1:18000 @2022-04-08 00:53:00.990000+00:00
	product#3302: 1:18000 @2022-04-08 00:52:45.459000+00:00



### Deleting a row

In [ ]:
print('Deleting a row by key')
key = 'order#2498'.encode()

row = table.direct_row(row_key)
row.delete()
row.commit()

In [ ]:
print('Getting all orders')

rows = table.read_rows()
for row in rows:
    print_row(row)

### Deleting table

In [ ]:
print('Deleting the {} table.'.format(TABLE_ID))
table.delete()